# Commodity Price Hedging for Oil & Gas Treasury

## Use Case Overview

**Business Problem:** Refiners face dual commodity exposure:
- **Input**: Crude oil purchases (Brent, WTI)
- **Output**: Refined products (gasoline, diesel, jet fuel)
- **Margin**: Crack spread = Product price - Crude price

**AI Applications:**
1. Price forecasting (ARIMA, LSTM)
2. Crack spread prediction
3. Optimal hedge timing
4. Sentiment analysis from news

---

## Learning Objectives

1. Analyze commodity price patterns
2. Calculate and forecast crack spreads
3. Build ARIMA forecasting model
4. Create hedge recommendation engine

In [ ]:
# Imports
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.metrics import mean_absolute_error, mean_squared_error

import sys
sys.path.append('..')
from src.treasury_sim.generators import generate_commodity_prices, set_seed

print("✅ Setup complete!")

## 1. Generate Commodity Data

In [ ]:
set_seed(42)

# Generate 2 years of commodity data
commodities, commodity_pivot = generate_commodity_prices(days=730, seed=42)

print(f"📊 Generated {len(commodities):,} price records")
print(f"\nCommodities: {commodities['commodity'].unique().tolist()}")

In [ ]:
# Preview
commodity_pivot.tail(10)

## 2. Crack Spread Analysis

### Thinking Trace 🧠

> **What is a crack spread?**
> - Refining margin = Revenue from products - Cost of crude
> - 3-2-1 crack: 3 barrels crude → 2 barrels gasoline + 1 barrel diesel
> - Key profit indicator for refiners

In [ ]:
# Calculate crack spreads (per barrel basis)
# Gasoline: ~42 gallons per barrel
# Diesel: ~42 gallons per barrel

commodity_pivot['GASOLINE_CRACK'] = commodity_pivot['GASOLINE'] * 42 - commodity_pivot['BRENT']
commodity_pivot['DIESEL_CRACK'] = commodity_pivot['DIESEL'] * 42 - commodity_pivot['BRENT']

# 3-2-1 crack spread
commodity_pivot['CRACK_321'] = (
    (2 * commodity_pivot['GASOLINE'] * 42 + 1 * commodity_pivot['DIESEL'] * 42) / 3 - commodity_pivot['BRENT']
)

print("📊 CRACK SPREAD STATISTICS")
print("=" * 50)
print(commodity_pivot[['GASOLINE_CRACK', 'DIESEL_CRACK', 'CRACK_321']].describe().round(2))

In [ ]:
# Visualize prices and spreads
fig = make_subplots(rows=2, cols=2,
    subplot_titles=('Crude Prices', 'Product Prices', '3-2-1 Crack Spread', 'Spread Distribution'))

# Crude prices
for crude in ['BRENT', 'WTI']:
    fig.add_trace(go.Scatter(x=commodity_pivot.index, y=commodity_pivot[crude], name=crude), row=1, col=1)

# Product prices  
for product in ['GASOLINE', 'DIESEL']:
    fig.add_trace(go.Scatter(x=commodity_pivot.index, y=commodity_pivot[product], name=product), row=1, col=2)

# Crack spread
fig.add_trace(go.Scatter(x=commodity_pivot.index, y=commodity_pivot['CRACK_321'], 
                         name='3-2-1 Crack', fill='tozeroy'), row=2, col=1)

# Distribution
fig.add_trace(go.Histogram(x=commodity_pivot['CRACK_321'], nbinsx=50, name='Distribution'), row=2, col=2)

fig.update_layout(height=600, title_text='Commodity Price Analysis')
fig.show()

## 3. Price Forecasting with ARIMA

In [ ]:
try:
    from statsmodels.tsa.arima.model import ARIMA
    ARIMA_AVAILABLE = True
    print("✅ ARIMA available")
except ImportError:
    ARIMA_AVAILABLE = False
    print("⚠️ Install statsmodels: pip install statsmodels")

In [ ]:
if ARIMA_AVAILABLE:
    # Prepare data
    brent = commodity_pivot['BRENT'].dropna()
    train = brent[:-30]  # Last 30 days for testing
    test = brent[-30:]
    
    # Fit ARIMA model
    print("🔧 Training ARIMA(5,1,2) model...")
    model = ARIMA(train, order=(5, 1, 2))
    fitted = model.fit()
    
    # Forecast
    forecast = fitted.forecast(steps=30)
    
    # Evaluate
    mae = mean_absolute_error(test, forecast)
    rmse = np.sqrt(mean_squared_error(test, forecast))
    mape = np.mean(np.abs((test - forecast) / test)) * 100
    
    print(f"\n📊 ARIMA FORECAST PERFORMANCE")
    print(f"MAE:  ${mae:.2f}")
    print(f"RMSE: ${rmse:.2f}")
    print(f"MAPE: {mape:.1f}%")

In [ ]:
if ARIMA_AVAILABLE:
    # Visualize forecast
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=train.index[-60:], y=train[-60:], name='Training', mode='lines'))
    fig.add_trace(go.Scatter(x=test.index, y=test.values, name='Actual', mode='lines'))
    fig.add_trace(go.Scatter(x=test.index, y=forecast.values, name='Forecast', 
                             mode='lines', line=dict(dash='dash')))
    
    fig.update_layout(title='Brent Crude Price Forecast (ARIMA)', height=400,
                      xaxis_title='Date', yaxis_title='Price ($/bbl)')
    fig.show()

## 4. Hedge Recommendation Engine

### Thinking Trace 🧠

> **When to hedge?**
> - **Contango** (future > spot): Consider buying futures
> - **Backwardation** (future < spot): Natural incentive to hedge
> - **High volatility**: Increase hedge ratio
> - **Margin pressure**: Lock in crack spreads

In [ ]:
def generate_hedge_recommendation(price_data, current_hedge_ratio=0.5):
    """
    Generate hedge recommendations based on market conditions.
    """
    latest = price_data.iloc[-1]
    avg_30d = price_data.iloc[-30:].mean()
    vol_30d = price_data.iloc[-30:].pct_change().std() * np.sqrt(252) * 100
    
    recommendations = []
    
    # Brent analysis
    brent_current = latest['BRENT']
    brent_avg = avg_30d['BRENT']
    brent_vol = vol_30d
    
    if brent_current > brent_avg * 1.05:
        rec = "🔴 INCREASE HEDGE - Prices above 30D average"
        target_ratio = min(current_hedge_ratio + 0.2, 1.0)
    elif brent_current < brent_avg * 0.95:
        rec = "🟢 REDUCE HEDGE - Prices below 30D average"
        target_ratio = max(current_hedge_ratio - 0.1, 0.3)
    else:
        rec = "🟡 MAINTAIN - Prices near average"
        target_ratio = current_hedge_ratio
    
    # Crack spread analysis
    crack = latest['CRACK_321']
    crack_avg = avg_30d['CRACK_321']
    
    if crack > crack_avg * 1.1:
        crack_rec = "🟢 Lock in favorable crack spread"
    elif crack < crack_avg * 0.9:
        crack_rec = "🔴 Margin pressure - consider inventory reduction"
    else:
        crack_rec = "🟡 Crack spread normal"
    
    return {
        'Brent Current': f"${brent_current:.2f}",
        'Brent 30D Avg': f"${brent_avg:.2f}",
        'Volatility (Ann)': f"{brent_vol:.1f}%",
        'Crude Recommendation': rec,
        'Target Hedge Ratio': f"{target_ratio:.0%}",
        'Crack Spread': f"${crack:.2f}",
        'Crack Recommendation': crack_rec
    }

# Generate recommendation
rec = generate_hedge_recommendation(commodity_pivot, current_hedge_ratio=0.6)

print("📊 HEDGE RECOMMENDATION")
print("=" * 60)
for k, v in rec.items():
    print(f"{k:25s}: {v}")

## 5. Executive Dashboard

In [ ]:
# Create executive dashboard
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Brent Price (1Y)', '3-2-1 Crack Spread', 
                   'Rolling 30D Volatility', 'Price-Crack Correlation'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"type": "scatter"}]]
)

# Brent with moving average
fig.add_trace(go.Scatter(x=commodity_pivot.index, y=commodity_pivot['BRENT'], name='Brent'), row=1, col=1)
fig.add_trace(go.Scatter(x=commodity_pivot.index, y=commodity_pivot['BRENT'].rolling(30).mean(), 
                         name='30D MA', line=dict(dash='dash')), row=1, col=1)

# Crack spread with bands
crack_mean = commodity_pivot['CRACK_321'].mean()
crack_std = commodity_pivot['CRACK_321'].std()
fig.add_trace(go.Scatter(x=commodity_pivot.index, y=commodity_pivot['CRACK_321'], 
                         name='Crack', fill='tozeroy'), row=1, col=2)
fig.add_hline(y=crack_mean, line_dash="dash", row=1, col=2)

# Volatility
rolling_vol = commodity_pivot['BRENT'].pct_change().rolling(30).std() * np.sqrt(252) * 100
fig.add_trace(go.Scatter(x=commodity_pivot.index, y=rolling_vol, name='Vol %'), row=2, col=1)

# Correlation scatter
fig.add_trace(go.Scatter(x=commodity_pivot['BRENT'], y=commodity_pivot['CRACK_321'], 
                         mode='markers', marker=dict(size=3), name='Corr'), row=2, col=2)

fig.update_layout(height=600, title_text='Commodity Hedging Dashboard', showlegend=False)
fig.show()

## 6. Key Takeaways

### AI Models for Commodity Hedging

| Model | Application | Accuracy |
|-------|-------------|----------|
| **ARIMA** | Short-term price direction | Medium |
| **LSTM** | Pattern recognition | Medium-High |
| **NLP/Sentiment** | News impact analysis | Supplementary |
| **RL** | Optimal execution timing | Research stage |

### Limitations

⚠️ **Important**: Commodity price prediction is notoriously difficult
- Markets are efficient (hard to beat)
- Geopolitical events are unpredictable
- Best use: Risk management, not speculation

---

*Author: Ozgur Guler (ozgur.guler1@gmail.com)*